In [13]:
import matplotlib.pyplot as plt

import os
import glob
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.transforms as T

In [2]:
class ForgeryClassificationDataset(Dataset):
    def __init__(self, authentic_dir, forged_dir, img_size=128):
        self.img_paths = []
        self.labels = []
        self.size = img_size

        # authentic = label 0
        for p in sorted(glob.glob(os.path.join(authentic_dir, "*.png"))):
            self.img_paths.append(p)
            self.labels.append(0)

        # forged = label 1
        for p in sorted(glob.glob(os.path.join(forged_dir, "*.png"))):
            self.img_paths.append(p)
            self.labels.append(1)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]

        # load + resize
        img = Image.open(img_path).convert("RGB").resize((self.size, self.size))
        img_np = np.array(img).astype(np.float32) / 255.0

        # to CHW tensor
        img_t = torch.tensor(img_np).permute(2, 0, 1)
        label_t = torch.tensor(label, dtype=torch.long)

        return img_t, label_t

In [3]:
class ClassifierCNN(nn.Module):
    """
    Baseline CNN classifier.
    """
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),       # 64x64

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),       # 32x32

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),       # 16x16
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, 2),      # authentic vs forged
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [4]:
def train_classifier():

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    ds = ForgeryClassificationDataset(
        authentic_dir="data/train_images/authentic",
        forged_dir="data/train_images/forged",
        img_size=128
    )

    loader = DataLoader(ds, batch_size=16, shuffle=True, num_workers=0)

    model = ClassifierCNN().to(device)
    opt = torch.optim.Adam(model.parameters(), lr=0.1)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(10):
        model.train()
        total_loss = 0
        correct = 0

        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            logits = model(imgs)
            loss = loss_fn(logits, labels)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            correct += (logits.argmax(dim=1) == labels).sum().item()

        acc = correct / len(ds)
        print(f"Epoch {epoch+1} | loss={total_loss/len(loader):.4f} | acc={acc:.3f}")

    torch.save(model.state_dict(), "baseline_classifier.pth")
    print("Saved baseline classifier.")

In [5]:
if __name__ == "__main__":
    train_classifier()

Using device: cpu
Epoch 1 | loss=1165.4872 | acc=0.512
Epoch 2 | loss=0.7012 | acc=0.513
Epoch 3 | loss=0.6976 | acc=0.509
Epoch 4 | loss=0.6964 | acc=0.529
Epoch 5 | loss=0.6999 | acc=0.504
Epoch 6 | loss=0.6943 | acc=0.520
Epoch 7 | loss=0.6982 | acc=0.513
Epoch 8 | loss=0.6977 | acc=0.516
Epoch 9 | loss=0.6952 | acc=0.526
Epoch 10 | loss=0.6988 | acc=0.507
Saved baseline classifier.


In [6]:
class ClassifierCNN_update(nn.Module):
    def __init__(self):
        super().__init__()

        # Feature extractor
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 128x128 -> 64x64 (example input)

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64x64 -> 32x32

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1, dilation=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1, dilation=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32 -> 16x16

            # Block 4 (optional, for capturing larger context)
            nn.Conv2d(128, 256, kernel_size=3, padding=2, dilation=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=2, dilation=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)  # 16x16 -> 8x8
        )

        # Classifier
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),  # output 256x1x1
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # authentic vs forged
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [7]:
def train_classifier_improved():

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    ds = ForgeryClassificationDataset(
        authentic_dir="data/train_images/authentic",
        forged_dir="data/train_images/forged",
        img_size=128
    )

    loader = DataLoader(ds, batch_size=16, shuffle=True, num_workers=0)

    model = ClassifierCNN_update().to(device)
    opt = torch.optim.Adam(model.parameters(), lr=0.1)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(10):
        model.train()
        total_loss = 0
        correct = 0

        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            logits = model(imgs)
            loss = loss_fn(logits, labels)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            correct += (logits.argmax(dim=1) == labels).sum().item()

        acc = correct / len(ds)
        print(f"Epoch {epoch+1} | loss={total_loss/len(loader):.4f} | acc={acc:.3f}")

    torch.save(model.state_dict(), "baseline_classifier.pth")
    print("Saved baseline classifier.")

In [8]:
if __name__ == "__main__":
    train_classifier_improved()

Using device: cpu
Epoch 1 | loss=0.9028 | acc=0.514
Epoch 2 | loss=0.6952 | acc=0.520
Epoch 3 | loss=0.6933 | acc=0.521
Epoch 4 | loss=0.6988 | acc=0.519
Epoch 5 | loss=0.6951 | acc=0.527
Epoch 6 | loss=0.6958 | acc=0.513
Epoch 7 | loss=0.6977 | acc=0.524
Epoch 8 | loss=0.6973 | acc=0.519
Epoch 9 | loss=0.6952 | acc=0.526
Epoch 10 | loss=0.6956 | acc=0.517
Saved baseline classifier.


In [15]:
class PatchFeatureExtractor(nn.Module):
    """
    Small CNN to extract features from each patch.
    """
    def __init__(self, in_channels=3, feature_dim=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1))
        )
        self.feature_dim = feature_dim
        self.fc = nn.Linear(64, feature_dim)

    def forward(self, x):
        # x: (batch, 3, patch_h, patch_w)
        x = self.conv(x)          # (batch, 64, 1, 1)
        x = x.view(x.size(0), -1) # (batch, 64)
        x = self.fc(x)            # (batch, feature_dim)
        x = F.normalize(x, dim=1) # normalize for similarity comparisons
        return x

class PatchBasedCopyMoveDetector(nn.Module):
    """
    Detects copy-move forgery by comparing patch features.
    """
    def __init__(self, patch_size=32, stride=16, feature_dim=64):
        super().__init__()
        self.patch_size = patch_size
        self.stride = stride
        self.feature_extractor = PatchFeatureExtractor(feature_dim=feature_dim)
        self.classifier = nn.Sequential(
            nn.Linear(feature_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # authentic vs forged
        )

    def extract_patches(self, x):
        """
        Extracts overlapping patches from input image.
        x: (batch, 3, H, W)
        returns: patches (num_patches_total, 3, patch_h, patch_w)
        """
        patches = x.unfold(2, self.patch_size, self.stride) \
                   .unfold(3, self.patch_size, self.stride)
        # patches: (batch, 3, num_h, num_w, patch_h, patch_w)
        batch, c, num_h, num_w, ph, pw = patches.shape
        patches = patches.permute(0,2,3,1,4,5).contiguous()
        patches = patches.view(-1, c, ph, pw)
        return patches, num_h, num_w

    def forward(self, x):
        # Extract patches
        patches, num_h, num_w = self.extract_patches(x)

        # Extract features for each patch
        patch_features = self.feature_extractor(patches)  # (num_patches_total, feature_dim)

        # Compute similarity map: naive example using mean features
        patch_features = patch_features.view(x.size(0), num_h*num_w, -1)
        mean_features = patch_features.mean(dim=1)  # (batch, feature_dim)

        # Classify based on aggregated patch features
        out = self.classifier(mean_features)
        return out

In [17]:
def train_classifier_patch():

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    ds = ForgeryClassificationDataset(
        authentic_dir="data/train_images/authentic",
        forged_dir="data/train_images/forged",
        img_size=128
    )

    loader = DataLoader(ds, batch_size=16, shuffle=True, num_workers=0)

    model = PatchBasedCopyMoveDetector().to(device)
    opt = torch.optim.Adam(model.parameters(), lr=0.1)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(10):
        model.train()
        total_loss = 0
        correct = 0

        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            logits = model(imgs)
            loss = loss_fn(logits, labels)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            correct += (logits.argmax(dim=1) == labels).sum().item()

        acc = correct / len(ds)
        print(f"Epoch {epoch+1} | loss={total_loss/len(loader):.4f} | acc={acc:.3f}")

    torch.save(model.state_dict(), "baseline_classifier.pth")
    print("Saved baseline classifier.")

In [21]:
if __name__ == "__main__":
    train_classifier_patch()

Using device: cpu
Epoch 1 | loss=0.6971 | acc=0.520
Epoch 2 | loss=0.6951 | acc=0.518
Epoch 3 | loss=0.6959 | acc=0.517
Epoch 4 | loss=0.6951 | acc=0.515
Epoch 5 | loss=0.6972 | acc=0.508
Epoch 6 | loss=0.6969 | acc=0.517
Epoch 7 | loss=0.6989 | acc=0.512


KeyboardInterrupt: 

In [32]:
class PatchFeatureExtractor(nn.Module):
    """
    Small CNN producing a feature vector for each patch.
    """
    def __init__(self, feature_dim=64):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1)),
        )
        self.fc = nn.Linear(64, feature_dim)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)   # (batch, 64)
        x = self.fc(x)
        x = F.normalize(x, dim=1)   # important for cosine similarity
        return x

# ---------------------------
# Full Copy-Move Model
# ---------------------------
class CopyMoveSelfSimilarity(nn.Module):
    """
    Patch-based model using self-similarity matrix for copy-move detection.
    """
    def __init__(self, patch_size=32, stride=16, feature_dim=64):
        super().__init__()
        self.patch_size = patch_size
        self.stride = stride
        self.feature_dim = feature_dim

        self.patch_net = PatchFeatureExtractor(feature_dim)

        # Classifier takes summary of similarity matrix
        self.classifier = nn.Sequential(
            nn.Linear(4, 32),
            nn.ReLU(),
            nn.Linear(32, 2)   # authentic vs forged
        )

    # ---------------------------
    # Patch extraction
    # ---------------------------
    def extract_patches(self, x):
        """
        x: (B, 3, H, W)
        returns:
            patches: (B * num_patches, 3, P, P)
            num_h, num_w: patch grid size
        """
        patches = x.unfold(2, self.patch_size, self.stride) \
                   .unfold(3, self.patch_size, self.stride)
        
        B, C, num_h, num_w, P1, P2 = patches.shape
        
        patches = patches.permute(0,2,3,1,4,5).contiguous()
        patches = patches.view(-1, C, P1, P2)

        return patches, num_h, num_w

    # ---------------------------
    # Forward pass
    # ---------------------------
    def forward(self, x):
        """
        x: (B, 3, H, W)
        """
        B = x.size(0)

        # 1. Extract patches
        patches, num_h, num_w = self.extract_patches(x)
        N = num_h * num_w   # patches per image

        # 2. Get features for each patch
        feat = self.patch_net(patches)    # (B*N, feature_dim)

        # 3. Reshape back per image
        feat = feat.view(B, N, self.feature_dim)  # (B, N, F)

        # 4. Compute self-similarity matrix for each image
        #    sim[b] = N x N matrix of cosine similarities
        sim = torch.bmm(feat, feat.transpose(1, 2))   # (B, N, N)

        # 5. Self-similarity statistics (summary)
        #    These patterns differ strongly between authentic and forged images
        max_sim = sim.max(dim=2).values.mean(dim=1)   # (B,)
        mean_sim = sim.mean(dim=(1,2))                # (B,)
        var_sim = sim.var(dim=(1,2))                  # (B,)
        topk_sim = torch.topk(sim.view(B, -1), 5, dim=1).values.mean(dim=1)

        # feature vector: [mean, max, variance, avg_top5]
        stats = torch.stack([mean_sim, max_sim, var_sim, topk_sim], dim=1)

        # 6. Classify (authentic vs forged)
        out = self.classifier(stats)

        return out   # also return similarity matrix for visualization

In [34]:
def train_classifier_patch_2():

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    ds = ForgeryClassificationDataset(
        authentic_dir="data/train_images/authentic",
        forged_dir="data/train_images/forged",
        img_size=128
    )

    loader = DataLoader(ds, batch_size=64, shuffle=True, num_workers=0)

    model = CopyMoveSelfSimilarity().to(device)
    opt = torch.optim.Adam(model.parameters(), lr=0.1)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(10):
        model.train()
        total_loss = 0
        correct = 0

        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            logits = model(imgs)
            loss = loss_fn(logits, labels)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            correct += (logits.argmax(dim=1) == labels).sum().item()

        acc = correct / len(ds)
        print(f"Epoch {epoch+1} | loss={total_loss/len(loader):.4f} | acc={acc:.3f}")

    torch.save(model.state_dict(), "baseline_classifier.pth")
    print("Saved baseline classifier.")

In [36]:
if __name__ == "__main__":
    train_classifier_patch_2()

Using device: cpu
Epoch 1 | loss=0.6956 | acc=0.519
Epoch 2 | loss=0.6959 | acc=0.517
Epoch 3 | loss=0.6975 | acc=0.514
Epoch 4 | loss=0.6952 | acc=0.527
Epoch 5 | loss=0.6970 | acc=0.514
Epoch 6 | loss=0.6963 | acc=0.527
Epoch 7 | loss=0.6969 | acc=0.517
Epoch 8 | loss=0.6953 | acc=0.521
Epoch 9 | loss=0.6955 | acc=0.519
Epoch 10 | loss=0.6972 | acc=0.513
Saved baseline classifier.
